In [1]:
import dash
from dash import html, dcc, Output, Input, State, ctx
import dash_bootstrap_components as dbc
from pymongo import MongoClient
import time
import uuid


In [2]:
import bcrypt

# Fonction pour hacher un mot de passe
def hash_password(password):
    # Générer un salt (clé aléatoire) et hacher le mot de passe
    salt = bcrypt.gensalt()
    hashed_password = bcrypt.hashpw(password.encode('utf-8'), salt)
    return hashed_password

# Fonction pour vérifier un mot de passe par rapport à un mot de passe haché
def check_password(password, hashed_password):
    # Comparer le mot de passe avec le haché stocké
    return bcrypt.checkpw(password.encode('utf-8'), hashed_password)


In [3]:
# Connexion à MongoDB
client = MongoClient('localhost', 27017)
# Sélectionner la base de données 'local'
db = client['local']
# Afficher les collections disponibles dans cette base de données
collections = db.list_collection_names()
print("Collections disponibles dans 'local' :", collections)
# Accéder à la collection 'users'
users_collection = db['Users']

# Lire tous les documents dans la collection 'users'
all_users = users_collection.find()
for user in all_users:
    print(user)

Collections disponibles dans 'local' : ['Users', 'Demands', 'startup_log']
{'_id': ObjectId('6706bc5ba5277fd2cd339d17'), 'First Name': 'Frank', 'Last Name': 'Durand', 'Profession': 'Engineer', 'Age': 64, 'Gender': 'Non-binary', 'Address': '116 Avenue Jean Jaurès, 92290 Châtenay-Malabry, France', 'Latitude': 48.751451, 'Longitude': 2.291243, 'Service Tag': "Garde d'animaux", 'Email': 'frankdurand@gmail.com', 'Password': '$2b$12$XE7t5yZtkxEXkU5wRIdUkOQaHXfRPxHSQw194c4VjvHnOQ5eCrjPm', 'Status': 'Gen+', 'Demands': [{'numero': 1, 'tag': "Garde d'animaux", 'description': "Service de Garde d'animaux", 'date': '2023-08-01', 'statut': 'actif'}, {'numero': 2, 'tag': "Garde d'animaux", 'description': "Service de Garde d'animaux", 'date': '2023-01-01', 'statut': 'terminé'}, {'numero': 3, 'tag': "Garde d'animaux", 'description': "Service de Garde d'animaux", 'date': '2024-04-23', 'statut': 'actif'}], 'Commentaires': ['Professionnel et efficace', 'Aucune remarque à faire, parfait.'], 'Points': 447}


In [4]:
# Obtenir les valeurs uniques du champ 'service tag'
users_collection = db['Users']
unique_service_tags = users_collection.distinct('Service Tag')

# Afficher les valeurs uniques
print("Valeurs uniques de 'service tag' :", unique_service_tags)

Valeurs uniques de 'service tag' : ['Aide au jardinage', 'Aide aux devoirs', 'Aide à la couture', 'Coaching sportif', 'Cours de musique', 'Covoiturage', 'Cuisine à domicile', 'Don de vêtements', 'Dépannage informatique', "Garde d'animaux", 'Partage de repas', 'Prêt de livres', 'Prêt de matériel de bricolage', "Réparation d'électroménagers", 'Travaux de peinture']


In [5]:
# Accéder à la collection 'Users'
users_collection = db['Users']

# Initialiser un ensemble (set) pour stocker les clés uniques
unique_keys = set()

# Parcourir les documents de la collection pour extraire les clés
for document in users_collection.find():
    unique_keys.update(document.keys())

# Afficher les clés uniques
print("Clés uniques dans la collection 'Users' :", unique_keys)

Clés uniques dans la collection 'Users' : {'Password', 'Address', 'Commentaires', 'Demands', 'Last Name', 'Services', 'Email', 'Age', 'Gender', 'First Name', 'Longitude', 'Latitude', 'Points', 'Status', '_id', 'Profile Image', 'Profession', 'Service Tag'}


In [6]:
# Récupérer tous les utilisateurs
all_users = users_collection.find()

# Hacher et mettre à jour les mots de passe
for user in all_users:
    if not user['Password'].startswith('$2b$'):  # Vérifie si le mot de passe n'est pas encore haché
        hashed_pw = bcrypt.hashpw(user['Password'].encode('utf-8'), bcrypt.gensalt())
        users_collection.update_one({'_id': user['_id']}, {'$set': {'Password': hashed_pw.decode('utf-8')}})
        print(f"Mot de passe pour {user['Email']} a été haché.")

In [11]:
from pymongo import MongoClient

# Connexion à MongoDB
client = MongoClient('localhost', 27017)
# Sélectionner la base de données 'local'
db = client['local']
# Accéder à la collection 'users'
users_collection = db['Users']
# Rechercher un utilisateur avec le prénom 'meriem'
user = users_collection.find_one({'First Name':'samar'})
# Vérifier si un utilisateur a été trouvé
if user:
    print(f"Utilisateur trouvé : {user}")
else:
    print("Aucun utilisateur trouvé avec le prénom 'samar'.")

Utilisateur trouvé : {'_id': ObjectId('6707d200e13481ee0f087b36'), 'First Name': 'samar', 'Last Name': 'samsam', 'Age': 30, 'Email': 'benmiloud@gmail.com', 'Password': '$2b$12$VtTiMQXhwGGx/1Er4nuPhObD7w9rZzMjZBE.DhGwTtZqSiY6qSWya', 'Address': '60 Avenue Léon Blum, 92290 Châtenay-Malabry, France', 'Latitude': 48.7709591, 'Longitude': 2.248999, 'Gender': 'Female', 'Status': 'GenDemandeur', 'Services': ['Prêt de matériel de bricolage', 'Prêt de livres'], 'Profession': 'datascientist', 'Profile Image': 'samar_samsam_profile.jpg', 'Demands': [{'numero': 6, 'tag': 'Don de vêtements', 'description': 'odifhbvvb', 'date': '2024-01-30', 'statut': 'actif'}]}


In [12]:
from pymongo import MongoClient
import pandas as pd
import matplotlib.colors as mcolors  # Pour générer une palette de couleurs

# Connexion à MongoDB
client = MongoClient('localhost', 27017)
db = client['local']
users_collection = db['Users']

# Récupérer les utilisateurs avec le statut 'GenDemandeur' et des demandes actives
users = list(users_collection.find({
    'Status': 'GenDemandeur', 
    'Demands.statut': 'actif'  # Filtrer uniquement les demandes avec statut 'actif'
}))

# Obtenir tous les Service Tags uniques
service_tags = set()  # Utiliser un set pour obtenir des valeurs uniques
for user in users:
    service_tags.add(user.get('Service Tag', 'Non spécifié'))
    for demande in user.get('Demands', []):
        service_tags.add(demande.get('tag', 'Non spécifié'))

# Générer des couleurs aléatoires pour chaque Service Tag
colors = list(mcolors.CSS4_COLORS)  # Utiliser les couleurs CSS pré-définies
service_tag_colors = {tag: colors[i % len(colors)] for i, tag in enumerate(service_tags)}

# Obtenir les informations des utilisateurs et de leurs demandes actives
user_data = []
demande_data = []
for user in users:
    user_data.append({
        "First Name": user.get('First Name', 'N/A'),
        "Last Name": user.get('Last Name', 'N/A'),
        "Service Tag": user.get('Service Tag', 'Non spécifié'),  # Utiliser 'Non spécifié' si le champ est manquant
        "Longitude": user.get('Longitude', 0.0),  # Mettre une valeur par défaut si les coordonnées sont manquantes
        "Latitude": user.get('Latitude', 0.0),
        "Color": service_tag_colors.get(user.get('Service Tag', 'Non spécifié'))  # Associer la couleur au Service Tag
    })
    
    # Parcourir les demandes pour récupérer uniquement celles avec statut 'actif'
    for demande in user.get('Demands', []):  # S'assurer que 'Demands' existe
        if demande.get('statut') == 'actif':
            demande_data.append({
                "First Name": user.get('First Name', 'N/A'),
                "Last Name": user.get('Last Name', 'N/A'),
                "Service Tag": demande.get('tag', 'Non spécifié'),
                "Description": demande.get('description', 'N/A'),
                "Date": demande.get('date', 'N/A'),
                "Lieu": demande.get('lieu', 'N/A'),
                "Color": service_tag_colors.get(demande.get('tag', 'Non spécifié'))  # Associer la couleur à la demande
            })

# Convertir en DataFrame pour la carte et le tableau
user_df = pd.DataFrame(user_data)
demande_df = pd.DataFrame(demande_data)

# Affichage des DataFrames pour vérifier les résultats
print("Users DataFrame:")
print(user_df.head())  # Affiche les premières lignes des utilisateurs avec les couleurs
print("\nDemandes DataFrame:")
print(demande_df.head())  # Affiche les premières lignes des demandes avec les couleurs

# Affichage du dictionnaire des couleurs
print("\nDictionnaire des couleurs pour chaque Service Tag:")
print(service_tag_colors)


Users DataFrame:
  First Name Last Name                    Service Tag  Longitude   Latitude  \
0    Charlie   Bernard  Prêt de matériel de bricolage   2.262378  48.766566   
1    Charlie   Lefevre                 Prêt de livres   2.267142  48.757015   
2      Diane    Moreau         Dépannage informatique   2.267142  48.757015   
3      Frank    Moreau         Dépannage informatique   2.252406  48.760962   
4      Alice   Lefevre              Aide au jardinage   2.256264  48.752885   

        Color  
0  blueviolet  
1  aquamarine  
2   burlywood  
3   burlywood  
4  chartreuse  

Demandes DataFrame:
  First Name Last Name                    Service Tag  \
0    Charlie   Bernard  Prêt de matériel de bricolage   
1    Charlie   Bernard  Prêt de matériel de bricolage   
2    Charlie   Bernard  Prêt de matériel de bricolage   
3    Charlie   Lefevre                 Prêt de livres   
4      Diane    Moreau         Dépannage informatique   

                                Description     

In [9]:
from bson.objectid import ObjectId

In [16]:
import dash
from dash import dcc, html, Input, Output, State, MATCH
import dash_bootstrap_components as dbc
from flask import Flask
from pymongo import MongoClient
import bcrypt
import random
import plotly.express as px
import plotly.io as pio
from geopy.geocoders import Nominatim
import base64
import os
import datetime

service_categories = [
    "Aide au jardinage",
    "Aide aux devoirs",
    "Aide à la couture",
    "Coaching sportif",
    "Cours de musique",
    "Covoiturage",
    "Cuisine à domicile",
    "Don de vêtements",
    "Dépannage informatique",
    "Garde d'animaux",
    "Partage de repas",
    "Prêt de livres",
    "Prêt de matériel de bricolage",
    "Réparation d'électroménagers",
    "Travaux de peinture"
]

# Configuration of the Dash app
external_stylesheets = [dbc.themes.BOOTSTRAP]
server = Flask(__name__)
app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)
app.config.suppress_callback_exceptions = True  # Suppress callback exceptions

# Connexion à MongoDB
client = MongoClient('localhost', 27017)
db = client['local']
users_collection = db['Users']

# Test de la connexion à MongoDB
try:
    client.admin.command('ping')
    print("Connexion à MongoDB réussie")
except Exception as e:
    print(f"Erreur de connexion à MongoDB : {e}")

# Styles personnalisés pour un design convivial
CONTENT_STYLE = {
    'backgroundColor': '#f1edea',
    'color': 'black',
    'min-height': '100vh',
    'padding': '20px',
    'fontFamily': 'Arial, sans-serif'
}

TITLE_STYLE = {
    'color': '#5062c6',  # Violet des titres
    'fontWeight': 'bold',
    'fontSize': '36px',
    'textAlign': 'center',
    'marginBottom': '30px'
}

BUTTON_STYLE = {
    'marginBottom': '10px',
    'fontSize': '16px',
    'backgroundColor': '#d3d3d3',  # Couleur par défaut des boutons
    'color': 'black',
    'border': '1px solid #d3d3d3'
}

UPLOAD_STYLE = {
    'width': '100%',
    'height': '60px',
    'lineHeight': '60px',
    'borderWidth': '1px',
    'borderStyle': 'dashed',
    'borderRadius': '5px',
    'textAlign': 'center',
    'margin': '10px 0',
    'color': 'black',
    'backgroundColor': '#f9f9f9'
}

LINK_STYLE = {
    'color': '#5062c6',
    'textDecoration': 'none',
    'fontWeight': 'bold'
}
"""dbc.NavLink("Dashboard", href="/dashboard", active="exact", style={
    'color': '#5062c6',
    'fontSize': '18px',
    'fontFamily': 'Arial, sans-serif',
    'padding': '10px',
    'textDecoration': 'none',
    'fontWeight': 'bold' if current_page == '/dashboard' else 'normal',
    'transition': 'transform 0.3s',
    'hover': {'transform': 'translateY(-2px)'}
})"""



# Geolocator pour convertir l'adresse en longitude et latitude
geolocator = Nominatim(user_agent="your_app_name_here")  # Remplacez par le nom de votre application

def get_lat_long(address):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Erreur lors du géocodage : {e}")
    return None, None

"""def serve_layout():
    return html.Div([
        dcc.Location(id='url', refresh=False),
        dcc.Store(id='user-email', data=None, storage_type='session'),

        html.Div(id='page-content', style=CONTENT_STYLE),
        # Composants cachés
        html.Div([
            # Composants de connexion manquants
            dbc.Input(id='login-email', placeholder='Email', type='email', style={'display': 'none'}),
            dbc.Input(id='login-password', placeholder='Mot de passe', type='password', style={'display': 'none'}),
            dbc.Button(id='login-button', style={'display': 'none'}),
            dbc.Button(id='logout-button', style={'display': 'none'}),
            html.Div(id='login-output', style={'display': 'none'}),
            dbc.Button(id='create-request-button', style={'display': 'none'}), 
            # Autres composants cachés si nécessaire
        ], style={'display': 'none'}),
    ])"""
def serve_layout():
    return html.Div([
        dcc.Location(id='url', refresh=False),
        dcc.Store(id='user-email', data=None, storage_type='session'),

        html.Div([
            html.Div(id='page-content', style=CONTENT_STYLE),
            # Hidden components inside 'page-content'
            html.Div([
                # Hidden components
                dbc.Input(id='login-email', placeholder='Email', type='email', style={'display': 'none'}),
                dbc.Input(id='login-password', placeholder='Mot de passe', type='password', style={'display': 'none'}),
                dbc.Button(id='login-button', style={'display': 'none'}),
                dbc.Button(id='logout-button', style={'display': 'none'}),
                html.Div(id='login-output', style={'display': 'none'}),
                dbc.Button(id='create-request-button', style={'display': 'none'}),
                # Other hidden components if needed
            ], style={'display': 'none'}),
        ])
    ])





# Layout de la page de connexion
login_layout = html.Div([
    dbc.Row([
        dbc.Col([
            html.H2("Connexion", style=TITLE_STYLE),
            dbc.Input(id="login-email", placeholder="Email", type="email", className="mb-2"),
            dbc.Input(id="login-password", placeholder="Mot de passe", type="password", className="mb-2"),
            dbc.Button("Se connecter", id="login-button", color="primary", className="mt-2",
                       style={'width': '100%', 'fontSize': '20px'}),
            html.Div(id="login-output", className="mt-3", style={'color': 'red'}),
            html.Div([
                html.P("Pas encore de compte ? ", style={'display': 'inline'}),
                dcc.Link('Inscrivez-vous ici', href='/register', style=LINK_STYLE)
            ], className="mt-3")
        ], width=6)
    ], justify='center')
], style=CONTENT_STYLE)
# Layout de la page d'inscription avec des boutons pour les services
register_layout = html.Div([
    dbc.Row([
        dbc.Col([
            html.H2("Inscription", style=TITLE_STYLE),
            dbc.Input(id="register-firstname", placeholder="Prénom", type="text", className="mb-2"),
            dbc.Input(id="register-lastname", placeholder="Nom", type="text", className="mb-2"),
            dbc.Input(id="register-age", placeholder="Âge", type="number", className="mb-2"),
            dbc.Input(id="register-email", placeholder="Email", type="email", className="mb-2"),
            dbc.Input(id="register-password", placeholder="Mot de passe", type="password", className="mb-2"),
            dbc.Input(id="register-address", placeholder="Adresse", type="text", className="mb-2"),
            dcc.Dropdown(
                id="register-gender",
                options=[
                    {'label': 'Homme', 'value': 'Male'},
                    {'label': 'Femme', 'value': 'Female'},
                    {'label': 'Non-binaire', 'value': 'Non-binary'}
                ],
                placeholder="Sélectionnez votre genre",
                className="mb-2",
                style={'backgroundColor': '#f9f9f9', 'color': 'black'}
            ),
            dcc.Dropdown(
                id="register-status",
                options=[
                    {'label': 'GenDemandeur', 'value': 'GenDemandeur'},
                    {'label': 'Gen+', 'value': 'Gen+'}
                ],
                placeholder="Sélectionnez votre statut",
                className="mb-4",
                style={'backgroundColor': '#f9f9f9', 'color': 'black'}
            ),
            html.H4("Sélectionnez les services désirés", className="mb-3", style={'color': '#5062c6'}),
            html.Div([
                # Générer les boutons pour les services
                *[dbc.Button(service_name, id=f"service-{i}", className="me-2 mb-2", style=BUTTON_STYLE) 
                for i, service_name in enumerate([
                    "Aide au jardinage", "Aide aux devoirs", "Aide à la couture", "Coaching sportif",
                    "Cours de musique", "Covoiturage", "Cuisine à domicile", "Don de vêtements",
                    "Dépannage informatique", "Garde d'animaux", "Partage de repas", "Prêt de livres",
                    "Prêt de matériel de bricolage", "Réparation d'électroménagers", "Travaux de peinture"
                ])]
            ], className="mb-4"),
            dbc.Input(id="register-profession", placeholder="Profession", type="text", className="mb-2"),
            # Composant de téléchargement d'image
            html.Div(id='image-upload-container', children=[
                dcc.Upload(
                    id='upload-image',
                    children=html.Div(['Glissez et déposez ou ', html.A('sélectionnez un fichier', style=LINK_STYLE)]),
                    style=UPLOAD_STYLE,
                    multiple=False
                ),
                html.Div(id='image-preview'),
                dbc.Button("Supprimer l'image", id='remove-image-button', color='danger', className='mt-2', style={'display': 'none'})
            ]),
            dbc.Button("S'inscrire", id="register-button", color="primary", className="mt-4", style={'width': '100%', 'fontSize': '20px'}),
            html.Div(id="register-output", className="mt-3", style={'color': 'red'}),
            html.Div([
                html.P("Déjà un compte ? ", style={'display': 'inline'}),
                dcc.Link('Connectez-vous ici', href='/', style=LINK_STYLE)
            ], className="mt-3")
        ], width=6)
    ], justify='center')
], style=CONTENT_STYLE)

# Ajout du composant Store pour stocker les services sélectionnés
register_layout.children.append(dcc.Store(id='selected-services', data=[]))
register_layout.children.append(dcc.Store(id='stored-image', data=None))

# Layout of the login page
login_layout = html.Div([
    dbc.Row([
        dbc.Col([
            html.H2("Connexion", style=TITLE_STYLE),
            dbc.Input(id="login-email", placeholder="Email", type="email", className="mb-2"),
            dbc.Input(id="login-password", placeholder="Mot de passe", type="password", className="mb-2"),
            dbc.Button("Se connecter", id="login-button", color="primary", className="mt-2", style={'width': '100%', 'fontSize': '20px'}),
            # Component for displaying login messages
            html.Div(id="login-output", className="mt-3", style={'color': 'red'}),
            html.Div([
                html.P("Pas encore de compte ? ", style={'display': 'inline'}),
                dcc.Link('Inscrivez-vous ici', href='/register', style=LINK_STYLE)
            ], className="mt-3")
        ], width=6)
    ], justify='center')
], style=CONTENT_STYLE)
   
def create_request_layout():
    service_categories = [
        "Aide au jardinage",
        "Aide aux devoirs",
        "Aide à la couture",
        "Coaching sportif",
        "Cours de musique",
        "Covoiturage",
        "Cuisine à domicile",
        "Don de vêtements",
        "Dépannage informatique",
        "Garde d'animaux",
        "Partage de repas",
        "Prêt de livres",
        "Prêt de matériel de bricolage",
        "Réparation d'électroménagers",
        "Travaux de peinture"
    ]

    return html.Div(
        [
            html.H2("Créer une demande", style=TITLE_STYLE),
            html.P("Veuillez remplir les informations ci-dessous pour créer une nouvelle demande.", className="text-muted mb-4"),
            
            # Titre de la demande
            dbc.Input(
                id='request-title',
                placeholder="Titre de la demande", 
                type="text", 
                className="mb-3"
            ),
            
            # Adresse
            dbc.Input(
                id='request-address',
                placeholder="Adresse", 
                type="text", 
                className="mb-3"
            ),
            
            # Description courte
            dbc.Input(
                id='request-short-description',
                placeholder="Description courte", 
                type="text", 
                className="mb-3"
            ),
            
            # Catégorie des services
            html.H6("Catégorie", className="mt-3"),
            html.Div(
                [
                    dbc.Button(
                        category_name,
                        id=f"category-{i}",
                        color="secondary",
                        outline=True,
                        className="me-2 mb-2"
                    )
                    for i, category_name in enumerate(service_categories)
                ],
                className="mb-4",
            ),
            
            # Plus de détails
            html.H6("Plus de détails"),
            dbc.Textarea(
                id='request-detailed-description',
                placeholder="Description détaillée", 
                className="mb-3"
            ),
            
            # Date et Heure de début
            dbc.Row(
                [
                    dbc.Col(
                        dbc.Input(
                            id='request-start-date',
                            placeholder="Date de début (JJ/MM/AAAA)", 
                            type="text", 
                            className="mb-3"
                        ),
                        width=6,
                    ),
                    dbc.Col(
                        dbc.Input(
                            id='request-start-time',
                            placeholder="Heure de début (HH:MM)", 
                            type="text", 
                            className="mb-3"
                        ),
                        width=6,
                    ),
                ]
            ),
            
            # Bouton de création de demande
            dbc.Button(
                "Créer la demande", 
                id="submit-request-button", 
                color="primary", 
                className="mt-3", 
                style={"width": "100%"}
            ),
            
            # Zone pour afficher les messages d'erreur ou de succès
            html.Div(id="create-request-output", className="mt-3", style={'color': 'red'}),

            # Stocker la catégorie sélectionnée
            dcc.Store(id='category-selected', data=None),
        ],
        style={"maxWidth": "500px", "margin": "auto", "paddingTop": "20px", "fontFamily": "Arial, sans-serif"}
    )

"""fig = px.scatter_mapbox(
    user_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="First Name",
    hover_data=["Service Tag"],
    color="Service Tag",
    color_discrete_map=service_tag_colors,
    zoom=12,
    height=500
)

# Personnaliser les icônes et le style de la carte
fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    showlegend=True
)"""
# Création de la carte avec les paramètres de base
fig = px.scatter_mapbox(
    user_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="First Name",
    hover_data=["Service Tag"],
    color="Service Tag",
    color_discrete_map=service_tag_colors,
    zoom=12,
    height=500
)

# Agrandir les points sur la carte
fig.update_traces(marker=dict(size=12))  # Ajustez la valeur de 'size' selon vos préférences

# Personnaliser le contenu de la bulle d'information pour supprimer la latitude et la longitude
fig.update_traces(
    hovertemplate="<b>%{hovertext}</b><br>" +
                  "Service Tag: %{customdata[0]}<extra></extra>"
)

# Styliser la bulle d'information et centrer le texte
fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=14,
        font_family="Arial",
        align="left"  # L'alignement 'center' n'est pas supporté, mais 'left' est par défaut
    )
)

# Mettre à jour le style de la carte et les marges
fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    showlegend=True
)




   
# Fonction pour le layout du tableau de bord
def dashboard_layout(user_email=None):
    if not user_email:
        return html.Div("Veuillez vous connecter pour voir le tableau de bord.")
    user = users_collection.find_one({'Email': user_email})
    if user:
        first_name = user.get('First Name', '')
        points = user.get('Points', 0)  # Assurez-vous que le champ 'Points' existe dans la base de données
        profile_image_filename = user.get('Profile Image', None)
        if profile_image_filename:
            profile_image_path = os.path.join('profile_images', profile_image_filename)
            if os.path.exists(profile_image_path):
                with open(profile_image_path, 'rb') as f:
                    profile_image_data = f.read()
                profile_image_src = 'data:image/png;base64,' + base64.b64encode(profile_image_data).decode('ascii')
            else:
                profile_image_src = 'https://via.placeholder.com/50x50.png'
        else:
            profile_image_src = 'https://via.placeholder.com/50x50.png'
    else:
        return html.Div("Utilisateur non trouvé.")
    
    return html.Div(
        [
            # En-tête avec les points et le message de bienvenue
            html.Div(
                [
                    dbc.Row(
                        [
                            dbc.Col(html.P(f"Points : {points}", className="text-muted"), width=9),
                            dbc.Col(
                                html.A(
                                    html.Img(
                                        src=profile_image_src, 
                                        className="rounded-circle", 
                                        style={"width": "50px"}
                                    ),
                                    href='/profile'
                                ),
                                width=3,
                                style={"textAlign": "right"}
                            ),
                        ],
                        align="center"
                    ),
                    html.H3(f"Bonjour {first_name} !", className="mt-2"),
                ],
                style={"padding": "20px", "backgroundColor": "#F1F8FA", "borderRadius": "10px", "marginBottom": "20px"}
            ),
            # Catégories des missions
            html.Div(
                [
                    html.H6("Catégories", className="mb-2"),
                    dbc.ButtonGroup(
                        [
                            dbc.Button("Covoiturage", outline=True, color="secondary", className="me-2"),
                            dbc.Button("Bricolage/jardinage", outline=True, color="secondary", className="me-2"),
                            dbc.Button("Enseignement", outline=True, color="secondary", className="me-2"),
                        ],
                        className="mb-3"
                    )
                ]
            ),
            # Missions récentes (sous forme de cartes)
            html.Div(
                [
                    dbc.Row(
                        [
                            dbc.Col(
                                dbc.Card(
                                    [
                                        html.Img(src="https://via.placeholder.com/150x100.png", className="card-img-top"),
                                        dbc.CardBody(
                                            [
                                                html.P("30/01/2025 20ptm", className="card-text text-muted"),
                                                html.H6("Moustache Bistro Pub", className="card-title"),
                                            ]
                                        )
                                    ],
                                    style={"width": "18rem"}
                                ),
                                width=6
                            ),
                            dbc.Col(
                                dbc.Card(
                                    [
                                        html.Img(src="https://via.placeholder.com/150x100.png", className="card-img-top"),
                                        dbc.CardBody(
                                            [
                                                html.P("11/03/2025 30ptm", className="card-text text-muted"),
                                                html.H6("Brasserie Les 3 Marcs", className="card-title"),
                                            ]
                                        )
                                    ],
                                    style={"width": "18rem"}
                                ),
                                width=6
                            )
                        ]
                    ),
                ],
                style={"marginBottom": "20px"}
            ),
            # Barre de recherche des missions
            html.Div(
                [
                    html.H6("Mission Demander", className="mt-4"),
                    dbc.InputGroup(
                        [
                            dbc.Input(placeholder="Chercher une mission", type="text"),
                            dbc.InputGroupText("🔍"),
                        ],
                        className="mb-4"
                    )
                ]
            ),
            # Aperçu d'une mission spécifique
            html.Div(
                dbc.Card(
                    [
                        html.Img(src="https://via.placeholder.com/300x150.png", className="card-img-top"),
                        dbc.CardBody(
                            [
                                html.P("5 rue de la république", className="card-text text-muted"),
                                html.H5("Bricolage", className="card-title"),
                                html.P("Je cherche une personne pour m'aider à monter un meuble IKEA.", className="card-text")
                            ]
                        )
                    ],
                    style={"width": "100%"}
                ),
                style={"marginBottom": "20px"}
            ),
                # Ajouter le graphique Mapbox en bas
            html.Div([
                dcc.Graph(
                    id='mapbox-graph',
                    figure=fig,
                    style={'width': '100%', 'height': '500px'}
                )
            ], style={
                'padding': '20px',
                'backgroundColor': '#f9f9f9'  # Ajustez la couleur pour correspondre à votre design
            }),
            
        ],
        style={"maxWidth": "500px", "margin": "auto", "paddingTop": "20px", "fontFamily": "Arial, sans-serif"}
    )
def create_navbar():
    navbar = dbc.Navbar(
        dbc.Container(
            [
                dbc.NavbarBrand("GenConnect+", href="/dashboard", style={
                    'color': '#5062c6',  # Violet des titres
                    'fontWeight': 'bold',
                    'fontSize': '28px',
                    'fontFamily': 'Arial, sans-serif'
                }),
                dbc.Nav(
                    [
                        dbc.NavItem(dbc.NavLink("Dashboard", href="/dashboard", style={
                            'color': '#5062c6',  # Couleur claire pour le texte
                            'fontSize': '18px',
                            'padding': '10px',
                            'fontFamily': 'Arial, sans-serif',
                            'fontWeight': 'bold',
                            'textDecoration': 'none'
                        })),
                        dbc.NavItem(dbc.NavLink("Profil", href="/profile", style={
                            'color': '#5062c6',  # Couleur claire pour le texte
                            'fontSize': '18px',
                            'padding': '10px',
                            'fontFamily': 'Arial, sans-serif',
                            'fontWeight': 'bold',
                            'textDecoration': 'none'
                        })),
                        dbc.NavItem(dbc.NavLink("Créer une demande", href="/create-request", style={
                            'color': '#5062c6',  # Couleur claire pour le texte
                            'fontSize': '18px',
                            'padding': '10px',
                            'fontFamily': 'Arial, sans-serif',
                            'fontWeight': 'bold',
                            'textDecoration': 'none'
                        })),
                        dbc.NavItem(dbc.NavLink("Se Déconnecter", href="http://127.0.0.1:3040/", style={
                            'color': 'red',  # Couleur claire pour le texte
                            'fontSize': '18px',
                            'padding': '10px',
                            'fontFamily': 'Arial, sans-serif',
                            'fontWeight': 'bold',
                            'textDecoration': 'none'
                        })),
                    ],
                    className="ml-auto", navbar=True
                ),
            ],
        ),
        color='#f1edea',  # Couleur de fond
        dark=True,  # Utilisation du texte clair
        style={
            'padding': '10px',  # Espacement autour de la barre
            'backgroundColor': '#f1edea',  # Correspondant au style
            'borderRadius': '15px',  # Bords arrondis
            'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)',  # Ombre légère
        },
        className="mb-5",  # Marge en bas pour espacer du contenu
    )
    return navbar




# Fonction pour le layout du profil
def profile_layout(user_email=None):
    if not user_email:
        return html.Div("Veuillez vous connecter pour voir votre profil.")

    # Récupérer les informations de l'utilisateur
    user = users_collection.find_one({'Email': user_email})
    if user:
        first_name = user.get('First Name', '')
        last_name = user.get('Last Name', '')
        age = user.get('Age', '')
        address = user.get('Address', '')
        gender = user.get('Gender', '')
        status = user.get('Status', '')
        profession = user.get('Profession', '')
        profile_image_filename = user.get('Profile Image', None)

        # Charger l'image de profil
        if profile_image_filename:
            profile_image_path = os.path.join('profile_images', profile_image_filename)
            if os.path.exists(profile_image_path):
                with open(profile_image_path, 'rb') as f:
                    profile_image_data = f.read()
                profile_image_src = 'data:image/png;base64,' + base64.b64encode(profile_image_data).decode('ascii')
            else:
                profile_image_src = 'https://via.placeholder.com/100x100.png'
        else:
            profile_image_src = 'https://via.placeholder.com/100x100.png'

        # Récupérer les demandes de l'utilisateur
        demands_collection = db['Demands']
        user_demands = list(demands_collection.find({'User Email': user_email}))

        # Générer la section des demandes
        demands_section = html.Div([
            html.H4("Mes Demandes", className="mt-4"),
            dbc.ListGroup([
                dbc.ListGroupItem([
                    html.Div([
                        html.H5(demand.get('title', 'Titre de la demande')),
                        html.P(f"Date : {demand.get('date', '')}"),
                        html.P(f"Description : {demand.get('short_description', '')}"),
                        dbc.Row([
                            dbc.Col(html.P("Statut :", className="mt-2"), width=2),
                            dbc.Col(
                                dbc.Switch(
                                    id={'type': 'demand-status-switch', 'index': str(demand.get('_id'))},
                                    label="Actif",
                                    value=(demand.get('status', '') == 'Open'),
                                    className="mt-2"
                                ), width=2
                            ),
                            dbc.Col(
                                html.Div(
                                    id={'type': 'demand-status-text', 'index': str(demand.get('_id'))},
                                    children=f"Statut : {demand.get('status', '')}"
                                ),
                                width=8
                            )
                        ])
                    ])
                ]) for demand in user_demands
            ])
        ])

        # Retourner le layout complet du profil
        return html.Div(
            [
                # Image de couverture
                html.Div(
                    [
                        html.Img(
                            src="https://via.placeholder.com/400x150.png",
                            style={"width": "100%", "borderTopLeftRadius": "10px", "borderTopRightRadius": "10px"},
                        )
                    ],
                    style={"overflow": "hidden", "borderRadius": "10px"}
                ),

                # Section avec l'image de profil et les informations de l'utilisateur
                html.Div(
                    [
                        # Image de profil
                        html.Div(
                            [
                                html.Img(
                                    src=profile_image_src,
                                    className="rounded-circle",
                                    style={
                                        "width": "80px",
                                        "border": "4px solid white",
                                        "marginTop": "-40px",
                                        "zIndex": "1"
                                    },
                                ),
                            ],
                            style={"display": "flex", "justifyContent": "center"},
                        ),

                        # Formulaire d'édition du profil
                        html.Div(
                            [
                                dbc.Input(
                                    id='profile-firstname',
                                    value=first_name,
                                    placeholder="Prénom",
                                    className="mt-2"
                                ),
                                dbc.Input(
                                    id='profile-lastname',
                                    value=last_name,
                                    placeholder="Nom",
                                    className="mt-2"
                                ),
                                dbc.Input(
                                    id='profile-age',
                                    value=age,
                                    placeholder="Âge",
                                    type="number",
                                    className="mt-2"
                                ),
                                dbc.Input(
                                    id='profile-address',
                                    value=address,
                                    placeholder="Adresse",
                                    className="mt-2"
                                ),
                                dcc.Dropdown(
                                    id='profile-gender',
                                    options=[
                                        {'label': 'Homme', 'value': 'Male'},
                                        {'label': 'Femme', 'value': 'Female'},
                                        {'label': 'Non-binaire', 'value': 'Non-binary'}
                                    ],
                                    value=gender,
                                    placeholder="Sélectionnez votre genre",
                                    className="mt-2",
                                ),
                                dcc.Dropdown(
                                    id='profile-status',
                                    options=[
                                        {'label': 'GenDemandeur', 'value': 'GenDemandeur'},
                                        {'label': 'Gen+', 'value': 'Gen+'}
                                    ],
                                    value=status,
                                    placeholder="Sélectionnez votre statut",
                                    className="mt-2",
                                ),
                                dbc.Input(
                                    id='profile-profession',
                                    value=profession,
                                    placeholder="Profession",
                                    className="mt-2"
                                ),
                                # Composant pour télécharger une nouvelle image de profil
                                dcc.Upload(
                                    id='upload-profile-image',
                                    children=html.Div(['Modifier la photo de profil']),
                                    style=UPLOAD_STYLE,
                                    multiple=False
                                ),
                                html.Div(id='profile-image-preview'),
                                dcc.Store(id='stored-profile-image', data=None),
                                dbc.Button(
                                    "Enregistrer les modifications",
                                    id='save-profile-button',
                                    color='primary',
                                    className='mt-4',
                                    style={'width': '100%'}
                                ),
                                html.Div(
                                    id='profile-save-output',
                                    className="mt-3",
                                    style={'color': 'red'}
                                ),
                                dbc.Button(
                                    "Créer une demande",
                                    id='create-request-button',
                                    color='primary',
                                    className='mt-4',
                                    style={'width': '100%'}
                                ),
                            ]
                        )
                    ],
                    style={
                        "position": "relative",
                        "backgroundColor": "#F7F8FA",
                        "paddingBottom": "20px",
                        "borderRadius": "10px"
                    }
                ),

                # Section des demandes de l'utilisateur
                demands_section,
            ],
            style={
                "maxWidth": "500px",
                "margin": "auto",
                "paddingTop": "20px",
                "fontFamily": "Arial, sans-serif"
            }
        )
    else:
        return html.Div("Utilisateur non trouvé.")

app.layout = serve_layout
app.validation_layout = html.Div([
    serve_layout(),
    login_layout,
    register_layout,
    dashboard_layout('example@example.com'),
    profile_layout('example@example.com'),
    create_request_layout(),
])
@app.callback(
    Output({'type': 'demand-status-text', 'index': MATCH}, 'children'),
    Input({'type': 'demand-status-switch', 'index': MATCH}, 'value'),
    State({'type': 'demand-status-switch', 'index': MATCH}, 'id'),
    prevent_initial_call=True
)
def update_demand_status(is_active, switch_id):
    demand_id = switch_id['index']
    new_status = 'Open' if is_active else 'Closed'
    # Mettre à jour la base de données
    demands_collection = db['Demands']
    demands_collection.update_one({'_id': ObjectId(demand_id)}, {'$set': {'status': new_status}})
    return f"Statut : {new_status}"

@app.callback(
    [Output(f'category-{i}', 'color') for i in range(len(service_categories))] +
    [Output('category-selected', 'data')],
    [Input(f'category-{i}', 'n_clicks') for i in range(len(service_categories))],
    prevent_initial_call=True
)
def select_category(*args):
    ctx = dash.callback_context
    if not ctx.triggered:
        # Couleurs par défaut
        return ['secondary'] * len(service_categories) + [dash.no_update]
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    colors = []
    selected_category = None
    for i, category_name in enumerate(service_categories):
        cat_id = f'category-{i}'
        if cat_id == button_id:
            colors.append('primary')  # Catégorie sélectionnée
            selected_category = category_name
        else:
            colors.append('secondary')  # Autres catégories
    return colors + [selected_category]

@app.callback(
    Output('create-request-output', 'children'),
    Input('submit-request-button', 'n_clicks'),
    State('request-title', 'value'),
    State('request-address', 'value'),
    State('request-short-description', 'value'),
    State('request-detailed-description', 'value'),
    State('request-start-date', 'value'),
    State('request-start-time', 'value'),
    State('user-email', 'data'),
    State('category-selected', 'data'),
    prevent_initial_call=True
)
def submit_request(n_clicks, title, address, short_desc, detailed_desc, start_date, start_time, user_email, selected_category):
    if n_clicks:
        if not all([title, address, short_desc, detailed_desc, start_date, start_time, selected_category]):
            return "Veuillez remplir tous les champs requis."
        
        # Vérifier le format de la date
        try:
            date_obj = datetime.datetime.strptime(start_date, "%d/%m/%Y")
            date_str = date_obj.strftime("%Y-%m-%d")
        except ValueError:
            return "Format de date invalide. Veuillez utiliser JJ/MM/AAAA."
        
        # Obtenir le numéro de demande (auto-incrément)
        requests_collection = db['Demands']
        last_demand = requests_collection.find_one(sort=[("numero", -1)])
        new_numero = (last_demand['numero'] + 1) if last_demand else 1

        # Créer la demande
        demand = {
            'User Email': user_email,
            'numero': new_numero,
            'tag': selected_category,
            'description': detailed_desc,
            'short_description': short_desc,
            'date': date_str,
            'time': start_time,
            'title': title,
            'address': address,
            'status': 'Open',
            'created_at': datetime.datetime.now()
        }

        # Insérer la demande dans la collection 'Demands'
        requests_collection.insert_one(demand)

        # Préparer l'entrée pour la liste 'Demands' de l'utilisateur
        demand_entry_for_user = {
            'numero': new_numero,
            'tag': selected_category,
            'description': detailed_desc,
            'date': date_str,
            'statut': 'actif'
        }

        # Mettre à jour la liste 'Demands' de l'utilisateur dans la collection 'Users'
        users_collection.update_one(
            {'Email': user_email},
            {'$push': {'Demands': demand_entry_for_user}}
        )

        return "Votre demande a été créée avec succès."

    return ""



# Callback pour afficher la page appropriée en fonction de l'URL et de l'état de l'utilisateur

@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname'),
    Input('user-email', 'data')],
    prevent_initial_call=True
)
def display_page(pathname, user_email):
    # Si l'utilisateur n'est pas connecté et n'est pas sur la page d'inscription, rediriger vers la page de connexion
    if not user_email and pathname != '/register':
        return login_layout

    # Créer le layout de la page avec une liste
    page_layout = []

    if pathname != '/':
        page_layout.append(create_navbar())

    # Ajouter le contenu de la page en fonction du chemin
    if pathname == '/register':
        page_layout.append(register_layout)
    elif pathname == '/dashboard':
        page_layout.append(dashboard_layout(user_email))
    elif pathname == '/profile':
        page_layout.append(profile_layout(user_email))
    elif pathname == '/create-request':
        page_layout.append(create_request_layout())
    else:
        # Par défaut, retourner la page de connexion si le chemin n'est pas reconnu
        return login_layout

    # Retourner le layout complet
    return html.Div(page_layout)
# @app.callback(
#     Output('url', 'pathname'),
#     Input('create-request-button', 'n_clicks'),
#     prevent_initial_call=True
# )
# def go_to_create_request(n_clicks):
#     if n_clicks:
#         return '/create-request'
#     return dash.no_update



# Callback combiné pour la connexion et la déconnexion
@app.callback(
    [Output('url', 'pathname'),
     Output('login-output', 'children'),
     Output('user-email', 'data')],
    [Input('login-button', 'n_clicks'),
     Input('logout-button', 'n_clicks'),
     Input('create-request-button', 'n_clicks')],
    [State('login-email', 'value'),
     State('login-password', 'value'),
     State('user-email', 'data')],
    prevent_initial_call=True
)

def login_logout(n_clicks_login, n_clicks_logout, n_clicks_create_request, email, password, user_email):
    ctx = dash.callback_context
    if not ctx.triggered:
        return dash.no_update, "", dash.no_update
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

        if button_id == 'login-button':
            # Existing login logic
            if n_clicks_login:
                if not email or not password:
                    return dash.no_update, "Veuillez entrer votre email et votre mot de passe.", dash.no_update
                user = users_collection.find_one({'Email': email})
                if user and bcrypt.checkpw(password.encode('utf-8'), user['Password'].encode('utf-8')):
                    return '/dashboard', "", email
                else:
                    return dash.no_update, "Email ou mot de passe incorrect.", dash.no_update
        elif button_id == 'logout-button':
            # Existing logout logic
            if n_clicks_logout:
                return '/', "", None
        elif button_id == 'create-request-button':
            if n_clicks_create_request:
                return '/create-request', "", dash.no_update

    return dash.no_update, "", dash.no_update


# Callback pour gérer l'inscription
@app.callback(
    Output('register-output', 'children'),
    Input('register-button', 'n_clicks'),
    State('register-firstname', 'value'),
    State('register-lastname', 'value'),
    State('register-age', 'value'),
    State('register-email', 'value'),
    State('register-password', 'value'),
    State('register-address', 'value'),
    State('register-gender', 'value'),
    State('register-status', 'value'),
    State('selected-services', 'data'),
    State('register-profession', 'value'),
    State('stored-image', 'data'),
    prevent_initial_call=True
)
def register_user(n_clicks, firstname, lastname, age, email, password, address, gender, status, services, profession, image_content):
    if n_clicks:
        if not firstname or not lastname or not email or not password or not age or not address:
            return "Veuillez remplir tous les champs requis."
        existing_user = users_collection.find_one({'Email': email})
        if existing_user:
            return "Un utilisateur avec cet email existe déjà."

        latitude, longitude = get_lat_long(address)
        if latitude is None or longitude is None:
            return "Adresse invalide. Veuillez entrer une adresse correcte."

        hashed_pw = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

        image_filename = None
        if image_content:
            _, content_string = image_content.split(',')
            image_data = base64.b64decode(content_string)
            image_filename = f"{firstname}_{lastname}_profile.jpg"
            if not os.path.exists('profile_images'):
                os.makedirs('profile_images')
            with open(os.path.join('profile_images', image_filename), 'wb') as f:
                f.write(image_data)

        # Mapping des IDs de services aux noms des services
        service_names = {
            f'service-{i}': service_name for i, service_name in enumerate([
                "Aide au jardinage", "Aide aux devoirs", "Aide à la couture", "Coaching sportif",
                "Cours de musique", "Covoiturage", "Cuisine à domicile", "Don de vêtements",
                "Dépannage informatique", "Garde d'animaux", "Partage de repas", "Prêt de livres",
                "Prêt de matériel de bricolage", "Réparation d'électroménagers", "Travaux de peinture"
            ])
        }

        selected_service_names = [service_names[service_id] for service_id in services]

        users_collection.insert_one({
            "First Name": firstname,
            "Last Name": lastname,
            "Age": age,
            "Email": email,
            "Password": hashed_pw.decode('utf-8'),
            "Address": address,
            "Latitude": latitude,
            "Longitude": longitude,
            "Gender": gender,
            "Status": status,
            "Services": selected_service_names,
            "Profession": profession,
            "Profile Image": image_filename
        })

        # Retourner un composant dcc.Location pour rediriger vers la page de connexion
        return dcc.Location(href='/', id='redirect-after-register')

    return ""


# Callback pour gérer la sélection et la désélection des services
@app.callback(
    [Output(f"service-{i}", "style") for i in range(15)] +
    [Output('selected-services', 'data')],
    [Input(f"service-{i}", "n_clicks") for i in range(15)],
    State('selected-services', 'data'),
    prevent_initial_call=True
)
def toggle_service(*args):
    n_clicks_list = args[:15]
    selected_services = args[15] if len(args) > 15 else []
    if not selected_services:
        selected_services = []
    ctx = dash.callback_context
    if not ctx.triggered:
        styles = [BUTTON_STYLE for _ in range(15)]
        return styles + [selected_services]

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Mise à jour des services sélectionnés
    if button_id in selected_services:
        selected_services.remove(button_id)
    else:
        selected_services.append(button_id)

    styles = []
    for i in range(15):
        if f"service-{i}" in selected_services:
            styles.append({
                'backgroundColor': '#3e4ba1',  # Violet plus foncé
                'color': 'white',
                'marginBottom': '10px',
                'fontSize': '16px',
                'border': '1px solid #3e4ba1'
            })
        else:
            styles.append(BUTTON_STYLE)
    return styles + [selected_services]

# Callback pour afficher l'aperçu de l'image téléchargée lors de l'inscription
@app.callback(
    [Output('image-preview', 'children'),
     Output('remove-image-button', 'style'),
     Output('stored-image', 'data')],
    [Input('upload-image', 'contents'),
     Input('remove-image-button', 'n_clicks')],
    [State('stored-image', 'data')],
    prevent_initial_call=True
)
def update_output(image_content, remove_clicks, stored_image):
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate

    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if triggered_id == 'upload-image' and image_content:
        return html.Img(src=image_content, style={'maxWidth': '100%', 'height': 'auto'}), {'display': 'block'}, image_content
    elif triggered_id == 'remove-image-button':
        return None, {'display': 'none'}, None
    else:
        return dash.no_update, dash.no_update, dash.no_update

# Callback pour mettre à jour l'image de profil lors du téléchargement dans le profil
@app.callback(
    [Output('profile-image-preview', 'children'),
     Output('stored-profile-image', 'data')],
    Input('upload-profile-image', 'contents'),
    prevent_initial_call=True
)
def update_profile_image(image_content):
    if image_content:
        return html.Img(src=image_content, style={'maxWidth': '100%', 'height': 'auto'}), image_content
    else:
        return dash.no_update, dash.no_update

# Callback pour enregistrer les modifications du profil
@app.callback(
    Output('profile-save-output', 'children'),
    Input('save-profile-button', 'n_clicks'),
    State('profile-firstname', 'value'),
    State('profile-lastname', 'value'),
    State('profile-age', 'value'),
    State('profile-address', 'value'),
    State('profile-gender', 'value'),
    State('profile-status', 'value'),
    State('profile-profession', 'value'),
    State('user-email', 'data'),
    State('stored-profile-image', 'data'),
    prevent_initial_call=True
)
def save_profile(n_clicks, firstname, lastname, age, address, gender, status, profession, email, image_content):
    if n_clicks:
        if not firstname or not lastname or not age or not address:
            return "Veuillez remplir tous les champs requis."
        update_fields = {
            "First Name": firstname,
            "Last Name": lastname,
            "Age": age,
            "Address": address,
            "Gender": gender,
            "Status": status,
            "Profession": profession,
        }
        if image_content:
            _, content_string = image_content.split(',')
            image_data = base64.b64decode(content_string)
            image_filename = f"{firstname}_{lastname}_profile.jpg"
            if not os.path.exists('profile_images'):
                os.makedirs('profile_images')
            with open(os.path.join('profile_images', image_filename), 'wb') as f:
                f.write(image_data)
            update_fields["Profile Image"] = image_filename
        users_collection.update_one({'Email': email}, {'$set': update_fields})
        return "Les modifications ont été enregistrées avec succès."
    return ""

if __name__ == '__main__':
    app.run_server(port=3040, debug=True)





Connexion à MongoDB réussie
